In [1]:
import pandas as pd
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils
url="train_set.csv"
dataset = pd.read_csv(url, sep='|', encoding='UTF-8')
dataset_len = len(dataset)

In [2]:
print(dataset.head(5))

         hyponim       hyponim1                       hyperonim  \
0  Капельмейстер  Капельмейстер    руководитель хоровой капеллы   
1  Капельмейстер  Капельмейстер                         дирижер   
2  Капельмейстер  Капельмейстер  руководитель военного оркестра   
3  Капельмейстер        дирижер                       профессия   
4  Капельмейстер      Профессия      род трудовой деятельности    

                                                text  \
0   Капельмейстер - руководитель хоровой капеллы ...   
1  КАПЕЛЬМЕ́ЙСТЕР, -а, м. Дирижер военного духово...   
2  Капельме́йстер (нем. Kapellmeister; перевод с ...   
3   Дирижер — профессия штучная, эксклюзивная, ко...   
4  Профе́ссия (лат. professio; от profiteor — «об...   

                                                link  bool  \
0  https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...   1.0   
1  https://kartaslov.ru/%D0%B7%D0%BD%D0%B0%D1%87%...   1.0   
2  https://kartaslov.ru/%D0%B7%D0%BD%D0%B0%D1%87%...   1.0   
3           

In [3]:
START = or_(rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN')),
        rule(gram('ADVB'))
    ).optional(),
    gram('NOUN'),
    or_(
        rule(gram('ADJF')).repeatable(),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')  
    ),
    rule(gram('NOUN')))

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),
    or_(
        eq('принадлежит')
    )
)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = or_(rule(
    eq(','),
    eq('а'),
    eq('также'),
    eq('весьма')
),
    rule(
    eq('и'),
    eq('тому'),
    eq('подобного')
    )
)


MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    ),
    rule(
        eq('-')
    ),
    rule(
        eq('–'),
        eq('это'),
        eq('не')
    ),
    rule(
    eq('является')
    ),
    rule(
        eq('-'),
        eq('это')
    ),
    rule(
    or_
        (eq('называют'),
        eq('называется'),
        eq('называли'))
    ),
    rule(
    eq('считается')
    ),
    rule(eq('то'),
        eq('есть')),
    rule(
    eq(','),
    eq('как')
    ),
    rule(
    eq('один'),
    eq('из')
    )
    ,
    rule(
    eq('.'),
    eq('описание'),
    eq(':')
    ),
    rule(
    eq('в'),
    eq('словаре'),
    eq('-')
    ),
    rule(
    eq('/'))
    
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(1,3)
    ), 
    rule(
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(    
        gram('ADVB'),
        eq('и'),
        gram('PRTF')
    #gram('NOUN').repeatable(1,3)
)
)


Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = or_(rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
        ))

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

RULE1 = rule(
    
    gram('ADVB'),
    eq('и'),
    gram('PRTF')
    #gram('NOUN').repeatable(1,3)
)

SHORT_WORD = or_(
    rule(
        eq(','),
        eq('-'),
        eq('а'),
        eq(','),
        eq('м'),
        eq('.')
    #, -а, м.
    )
)

ILI = or_(rule(
#или Ка́перцы, или Ка́порцы (лат. Cāpparis) —
    eq(','),
    eq('или'),
    gram('NOUN'),
    eq(','),
    eq('или'),
    gram('NOUN')
),
    rule(
    eq('или'),
    gram('NOUN'),
    gram('ADJF')
          ),
    rule(eq('и'), 
    gram('NOUN'))
         )

#или белянка капустная

In [4]:
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()
def plur(word):
    try:
        b = morph.parse(word)[0]
        c = b.inflect({'plur'}) 
        return (c.word)
    except: 
        return (word)

In [5]:
def delete_pros(text):
    array_with_accent = ['А́ ','а́', 'Е́ ','е́', 'О́ ','о́', 'У́ ','у́', 'И́ ','и́', 'Ю́ ','ю́', 'Я́ ','я́','е́']
    array_without_accent = ['А', 'а', 'Е', 'е', 'О', 'о', 'У', 'у', 'И', 'и', 'Ю', 'ю', 'Я', 'я','e']
    #for char in text:
    for i in range(len(array_with_accent)):
        text = text.replace(array_with_accent[i], array_without_accent[i])
    return text

In [6]:
a = 'капельме́йстер, -а, м. дирижер военного духового оркестра.  устар. вообще дирижер. дирижировал артур никиш, молодой и в высшей степени даровитый капельмейстер лейпцигской оперы. а. бородин, письмо ц. а. кюи, 12 июня  1881.'
print(a)
print(delete_pros(a))

капельме́йстер, -а, м. дирижер военного духового оркестра.  устар. вообще дирижер. дирижировал артур никиш, молодой и в высшей степени даровитый капельмейстер лейпцигской оперы. а. бородин, письмо ц. а. кюи, 12 июня  1881.
капельмейстер, -а, м. дирижер военного духового оркестра.  устар. вообще дирижер. дирижировал артур никиш, молодой и в высшей степени даровитый капельмейстер лейпцигской оперы. а. бородин, письмо ц. а. кюи, 12 июня  1881.


In [7]:
import numpy as np
import collections
count = []
def get_hyperonyms_rw1(i): 
    bool_count = 0
    word = dataset['hyponim1'][i]
    array_of_main_words = []
    array_of_main_words.append(word.lower())
    if (plur(word) != word.lower()):
        array_of_main_words.append(plur(word))
    array_of_hyper = []
    for main_word in array_of_main_words:    
        #HYPONYM = eq(utils.deaccent(main_word))
        #if (not(re.search(r'\й', main_word))):
        #    HYPONYM = eq(utils.deaccent(main_word))
        #else:
        #    HYPONYM = eq(main_word)
        HYPONYM = delete_pros(main_word)
        RULE = or_(
            rule(HYPONYM, ATAKJE, START, MID, END),
            rule(HYPONYM, MID, END),
            rule(START_S, END, KAK, HYPONYM),
            rule(END, INCLUDING, HYPONYM),
            rule(HYPONYM, MID, END, END),
            rule(HYPONYM, MID, START, END),
            rule(START, MID, HYPONYM),
            rule(HYPONYM, END),
            rule(HYPONYM, MID, RULE1),
            rule(HYPONYM, RULE1),
            rule(HYPONYM, SHORT_WORD, END),
            rule(gram('NOUN'), HYPONYM, gram('ADVB'), gram('ADJF')),
            rule(HYPONYM, ILI, MID, END),
            rule(HYPONYM, eq('и'), gram('NOUN'), eq(','), MID, eq('-'), END, END)
            )
        parser = Parser(RULE) 
        text = (dataset['text'][i]).lower()
        text = delete_pros(text)
        text = re.sub(r'\(.+?\)', '', text)
        text = re.sub(r'([\(\[]).*?([\)\]])','', text)
        text = text.lower().replace('* сергии радонежскии* ', '')
        #print(text)
        for idx, match in enumerate(parser.findall(text.lower())):
            k = [_.value for _ in match.tokens]                
            array_of_hyper.append(k)
    if (len(array_of_hyper) > 0):
        count.append(1)
        print(text)
        print(array_of_hyper)
        if(dataset['bool'][i] == 0):
            print('this word was not find in context as a pair "hyponym + hyperonim", there is a reason why')
            print(dataset['comment'][i])
    print('########################################')

        
        

In [8]:
i = 41
print(dataset['text'][i])
#print(delete_pros1(dataset['text'][i]))
get_hyperonyms_rw1(i)

Ка́рбас, также карба́с[1] и карбус, (вепс. karbaz) — парусно-гребное промысловое и транспортное судно среднего размера, одно из основных у поморов и распространённое у других жителей Севера и Сибири, начиная не позднее чем с XV[2] и до середины XX века. Для поморского диалекта русского языка характерно произношение с ударением на первый слог и форма множественного числа «карбаса́»[3].
карбас, также карбас и карбус,  — парусно-гребное промысловое и транспортное судно среднего размера, одно из основных у поморов и распространённое у других жителей севера и сибири, начиная не позднее чем с xv и до середины xx века. для поморского диалекта русского языка характерно произношение с ударением на первый слог и форма множественного числа «карбаса».
[['карбас', 'и', 'карбус', ',', '—', 'парусно', '-', 'гребное', 'промысловое', 'и', 'транспортное', 'судно']]
########################################


In [9]:
number_of_words = 0
for i in range(len(dataset)):
    get_hyperonyms_rw1(i)
    


 капельмейстер - руководитель хоровой капеллы и/или оркестра. военный капельмейстер — дирижёр полкового, эскадронного и тому подобного оркестра.
[['капельмейстер', '-', 'руководитель', 'хоровой', 'капеллы', 'и'], ['капельмейстер', '—', 'дирижёр']]
########################################
капельмейстер, -а, м. дирижер военного духового оркестра.  устар. вообще дирижер. дирижировал артур никиш, молодой и в высшей степени даровитый капельмейстер лейпцигской оперы. а. бородин, письмо ц. а. кюи, 12 июня  1881.
[['капельмейстер', ',', '-', 'а', ',', 'м', '.', 'дирижер', 'военного', 'духового', 'оркестра'], ['капельмейстер', 'лейпцигской', 'оперы']]
########################################
капельмейстер  — руководитель хоровой капеллы и/или оркестра.
[['капельмейстер', '—', 'руководитель', 'хоровой', 'капеллы', 'и']]
########################################
 дирижер — профессия штучная, эксклюзивная, которой научить практически нельзя. дирижером надо родиться. к этой профессии идут годами.
[[

[['каравелла', 'является', 'самолет', 'средней', 'дальности'], ['каравеллы', 'в', 'словаре', '-', 'это', 'быстрый', 'корабль']]
########################################
другим определением каравелла является самолет средней дальности.
[['каравелла', 'является', 'самолет', 'средней', 'дальности']]
########################################
 каравелла — тип парусного судна, распространённый в европе, особенно в португалии и испании, во второй половине xv — начале xvii века. один из первых и наиболее известный тип кораблей, с которых начиналась эпоха великих географических открытий.
[['каравелла', '—', 'тип', 'парусного', 'судна']]
########################################
 каравелла - европейский автомобиль.
[['каравелла', '-', 'европейский', 'автомобиль']]
this word was not find in context as a pair "hyponym + hyperonim", there is a reason why
Написала каравелла русскими буквами
########################################
 каракатицы  — отряд моллюсков из класса головоногих.
[['каракатицы', '

дилатационная кардиомиопатия - нарушение функционирования миокарда, приводящее к сердечной недостаточно­сти, при которой преобладают дилатация желудочков и систолическая дисфункция. симптомы включают одышку, усталость и периферические отеки. диагноз устанавливают на основании клинических данных и повышенного содержания натрийуретических пептидов, результатов рентгенографии грудной клетки, эхокардиографии и мрт. лечение ориентировано на устранение причины заболевания. если сердечная недостаточность является тяжелой и прогрессирующей, могут быть показаны: ресинхронизирующая сердечная терапия, икд, ликвидация тяжелой митральной регургитации или трансплантация сердца.
[['кардиомиопатия', '-', 'нарушение', 'функционирования', 'миокарда']]
########################################
кардиомиопатия - гетерогенная группа заболеваний миокарда, связанных с механической или электрической дисфункцией, которая обычно проявляется неадекватной гипертрофией или дилатацией. кардиомиопатии могут как изолир

########################################
кастелянша это такая профессия, принято считать, что этот человек отвечает за одежду или бельё.
[['кастелянша', 'это', 'такая', 'профессия']]
########################################
кастелянша считается женским родом деятельности
[['кастелянша', 'считается', 'женским', 'родом', 'деятельности']]
########################################
кастелянша / сотрудник бельевой компании
[['кастелянша', '/', 'сотрудник', 'бельевой', 'компании']]
########################################
########################################
кастомизация  — индивидуализация продукции под заказы конкретных потребителей путём внесения конструктивных или дизайнерских изменений . например, нанесение рисунка на приобретаемую в интернет-магазине футболку по заявке покупателя или изготовление свадебного торта с поздравительной надписью.
[['кастомизация', '—', 'индивидуализация', 'продукции', 'под', 'заказы', 'конкретных', 'потребителей', 'путём', 'внесения']]
####################

прогулочный каякинг, он же морской каякинг  — это путешествия по озёрам и морям, равнинным рекам и водохранилищам.
[['каякинг', '—', 'это', 'путешествия']]
########################################
каякинг - разновидность водного туризма
[['каякинг', '-', 'разновидность', 'водного', 'туризма']]
########################################
каякинг - это движение в бурном потоке воды.
[['каякинг', '-', 'это', 'движение', 'в', 'бурном', 'потоке', 'воды']]
########################################
